In [ ]:
%pip install xnat

In [ ]:
import xnat
from datetime import date
import os 

"""
INSTRUCTIONS: 
Edit the 'To Edit' section vars only. This is a single scan uploader you cannot upload multi scan data, if you would like to do that please use the batch uploader tool within the xnat UI for fNIRS. It is recommended you follow BIDS structure and XNAT subject/session/scan naming convensions used for the multi fNIRS multi subject uploader for consistancy and usability with other containers. NOTE: Do not include any space or special chars within file names/folder names or xnat component names e.g session, subject, project 
"""


#TO EDIT:
#Do not include '/' after the .org url e.g use 'https://demo.xnat.org' and NOT 'https://demo.xnat.org/'
xnatURL="https://your.xnat.org"
user="username"
password="Password"

#ProjectToAdd information to (USE YOUR PROJECT ID) if you do not know this you can check on OXI for the name
project="projectName"
#Subject To add to/create: 
subject="subjectName"
#Session To add to/create: 
sessionname="sessionName"
#Scan To add to/create: 
scan="scanName"
#pathtoFolderContainingFiles NOTE: If you are on windows change all '\' to '/'
pathToData="/path/to/your/single/scan/nirs/folder"

#DO NOT EDIT BELOW
#create session to connect
session = xnat.connect(xnatURL, user, password)

today=date.today().strftime("%y/%m/%d")

files = os.listdir(pathToData)

def get_wrapper_id(session, xnat_host, command_name, wrapper_name):
    # Send a GET request to the /xapi/commands endpoint
    response = session.get(f'{xnat_host}/xapi/commands')

    # Check the status code of the response
    if response.status_code != 200:
        raise Exception(f'Failed to get commands from XNAT at {xnat_host} with status code {response.status_code}')

    # Parse the JSON response
    commands = response.json()

    # Iterate over the commands
    for command in commands:
        # Check if the command name and wrapper name match the given parameters
        if command['name'] == command_name:
            # Iterate over the command's wrappers
            for wrapper in command['xnat']:
                # Check if the wrapper name matches the given parameter
                if wrapper['label'] == wrapper_name:
                    # If they match, return the wrapper ID
                    return wrapper['id']
    # If no matching command is found, raise an exception
    raise Exception(f'No command found with name {command_name} and wrapper name {wrapper_name}')

#create a subject 
SubjectPUT='{myXNAT}/data/projects/{projectName}/subjects/{subjectName}'.format(myXNAT=xnatURL, projectName=project, subjectName=subject)
session.put(SubjectPUT)

#create fNIRS Session
sessionNameForXNAT=subject+"_"+sessionname
SessionPUT='{myXNAT}/data/projects/{projectName}/subjects/{subjectName}/experiments/{sessionName}?fnirs:fnirsSessionData/label={label}'.format(myXNAT=xnatURL, projectName=project, subjectName=subject, sessionName=sessionNameForXNAT, date=today,label=sessionNameForXNAT)
session.put(SessionPUT)

#create fNIRS Scan
ScanPUT='{myXNAT}/data/projects/{projectName}/subjects/{subjectName}/experiments/{sessionName}/scans/{scanName}?xsiType=fnirs:fnirsScanData&fnirs:fnirsScanData/type=FNIRS&quality=usable'.format(myXNAT=xnatURL, projectName=project, subjectName=subject, sessionName=sessionNameForXNAT,scanName=scan)
session.put(ScanPUT)

# #create fNIRS Scan Resource folder (FNIRS Folder)
try:
    screateFolderPUT="{myXNAT}/data/projects/{projectName}/subjects/{subjectName}/experiments/{sessionName}/scans/{scanName}/resources/FNIRS?xsiType=xnat:resourceCatalog&format=FNIRS&content=FNIRS".format(myXNAT=xnatURL, projectName=project, subjectName=subject, sessionName=sessionNameForXNAT,scanName=scan)
    session.put(screateFolderPUT)
except:
    print("Folder already exists moving on")

#upload fNIRS Scan data
for file in files: 
    if ".DS_Store" not in file:
        print("Uploading... ", file)
        try:
            fileLocationToUpload="/data/projects/{projectName}/subjects/{subjectName}/experiments/{sessionName}/scans/{scanName}/resources/FNIRS/files/{fileName}".format(projectName=project,subjectName=subject, sessionName=sessionNameForXNAT, scanName=scan, fileName=file)
            filePath=os.path.join(pathToData,file)
            session.upload_file(uri=fileLocationToUpload, path=filePath)
        except Exception as error:
            print("An exception occurred:", error)

print("Done uploading files")

print("Launching convert2ndot on data")
contaainerID=get_wrapper_id(session, xnatURL, "neurodot-convert2ndot", "convert2ndot")


session.disconnect()